The last thing I was working on was:
* modifying the advect_layers function to stop layers from going crazy as they reach the edges of the domain (hopefully done)
* adjusting the bounds around which the main ODE is solved to end when layers reach this boundary area
* adapting all the analysis code to not query the ODE solutions past where they are valid

After messing with this a bit, potentially none of this is necessary. Instead, I added a maximum step size (in time) between reinterpolating the layer tracer points. This seems to resolve the stability issues I was seeing.

The logistic basal velocity field seems more reasonable. Need to play with it some more, but 30 and 50 m/yr max basal velocity seem to both produce reasonable results. Hopefully can use layers from 30 in the 50 simulation to capture Dusty's speed up idea.

Need to figure out how to calculate du/dz under the zero slope approximation.

Decided it made more sense to just compare vertical velocities directly. Need to figure out why the vertical velocity estimate from the ODEs is consistently slightly off. Error plot has a weird pattern. Then maybe add in a "layer direction represents flow direction" assumption to test in addition to zero slope.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy
from sympy import *
import pickle
import datetime
from tqdm import tqdm
import scipy.constants
import time

from flowline_ode.plots_setup import *
from flowline_ode.sia_model import *
from flowline_ode.finite_differences import *
from flowline_ode.ode_solve import *

In [ ]:
t_start_notebook = time.time()

### Problem setup

In [ ]:
n = 3.0 # Flow exponent for the actual model

output_results_base = f"outputs/{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_n{n:.1f}"
print(f"{output_results_base}")

In [ ]:
# Domain size
domain_x = 100000 # meters # 100 km for all other examples, 200 km for the new one
domain_z = 3000 # meters

# Grids for when discretization is needed
dx = 100
dz = 25
xs = np.arange(0, domain_x, dx)
zs = np.arange(0, domain_z, dz)

# Sympy symbolic variables
x = sympy.symbols('x', real=True, positive=True)
z = sympy.symbols('z', real=True, positive=True)

# Define surface geometry
surface_sym = domain_z - ((x / 18000.0)**3.0) # Rheology example

surface = sympy.lambdify(x, surface_sym, modules='numpy')


# Use sympy to build a function for the derivative of the surface
ds_dx_sym = sympy.diff(surface_sym, x)

ds_dx_lambdify = sympy.lambdify(x, ds_dx_sym, modules='numpy')
def ds_dx(x):
    tmp = ds_dx_lambdify(x)
    if np.isscalar(tmp):
        return tmp + (0*x)
    else:
        return tmp

# Plot the surface and its derivative with twin x axes
fig, ax = plt.subplots(figsize=(8, 4))
line_surf = ax.plot(xs/1e3, surface(xs), 'blue', label='s(x) [m]')
ax.tick_params(axis='y', colors='blue')
ax.set_ylabel('Surface elevation [m]', color='blue')
ax_right = ax.twinx()
ax_right.tick_params(axis='y', colors='red')
line_slope = ax_right.plot(xs/1e3, (180/np.pi) * np.tan(ds_dx(xs)), 'r--', label='ds_dx(s) [deg]')
ax_right.set_ylabel('Surface slope [deg]', color='red')

lns = line_surf + line_slope
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='upper right')

ax.set_title('Surface and surface slope')
ax.set_xlabel('Distance [km]')
ax.grid(True, axis='x')
fig.savefig(f"{output_results_base}_surface_and_slope.png")
plt.show()

### Generate SIA-based velocity field

Our SIA model is fully specified by a surface contour (including its derivative) and a basal velocity field. (Plus some constants, but we'll assume those don't change.)

We'd like to generate multiple examples with varying rheology (in this case defined as varying flow exponents) but identical surface velocity fields. In order to do this, we run the SIA model three times:
* First with a "reference n" flow exponent (that must be the lowest value we want to use) and zero basal velocity
* Again with the intended flow exponent and zero basal velocity
* And finally with the intended flow exponent and a basal velocity field calculated to compensate for the difference between the surface velocities of the two prior models.

This means that if we change nothing except for `n` between runs, we get identical surface velocity fields but varying englacial velocities.

In [ ]:
reference_n = 2.0 # Flow exponent for the reference model -- 2.0 for all rheologyexamples

assert n >= reference_n, ("The reference model flow exponent must be <= n or "
                          "else negative horizontal basal velocities are "
                          "required to make the surface velocities match.")

# Depth-dependent enhancement factor
enhancement_factor_sym = 1 # 2 + sympy.tanh((z - 1500)/100) # TODO ENHANCEMENT FACTOR

tstamp = datetime.datetime.now()

# Reference run
u_ref, _, _ = sia_model(x, z, surface_sym, ds_dx_sym, n=reference_n)
print(f"Time to run the reference SIA model: {datetime.datetime.now() - tstamp}")
tstamp = datetime.datetime.now()
# Run with intended flow exponent to figure out the needed basal velocity
u_test, w_test, du_dx_test = sia_model(x, z, surface_sym, ds_dx_sym, n=n,
                         enhancement_factor=enhancement_factor_sym) # TODO ENHANCEMENT FACTOR
basal_velocity = u_ref.subs(z, surface_sym) - u_test.subs(z, surface_sym)
print(f"Time to run the test SIA model: {datetime.datetime.now() - tstamp}")
tstamp = datetime.datetime.now()

# The actual model we'll use going forward
u, w, du_dx = sia_model(x, z, surface_sym, ds_dx_sym, n=n,
                        basal_velocity_sym=basal_velocity,
                        enhancement_factor=enhancement_factor_sym) # TODO ENHANCEMENT FACTOR

print(f"Time to run final SIA model: {datetime.datetime.now() - tstamp}")

In [ ]:
# z_vals = np.linspace(0, domain_z, 100)

# plt.plot(zs, sympy.lambdify(z, enhancement_factor_sym)(zs))
# plt.xlabel('z')
# plt.ylabel('enhancement_factor_sym')
# plt.title('Enhancement Factor')
# plt.show()

In [ ]:
def lambdify_and_vectorize_if_needed(vars, symbolic_expr, warn_if_vectorizing = True, **kwargs):
    """
    Acts just like sympy.lambdify() except also calls np.vectorize() on the
    result if the function lambdify() returns cannot accept vector inputs.
    (This happens if the returned expression contains an integral. It is not
    clear to me if this is a bug or the intended behavior. Always applying
    np.vectorize() isn't a good approach because the result is much slower
    than the natively vectorized functions from lambdify.)
    """

    l = sympy.lambdify(vars, symbolic_expr, **kwargs)

    # Figure out how many variables the lambdified function expects
    # vars may be either a single Symbol or a list of Symbols
    n_vars = 1
    try:
        n_vars = len(vars)
    except TypeError:
        n_vars = 1

    test_input = [np.array([1, 1]) for i in range(n_vars)]
    must_vectorize = False
    try:
        test_output = l(*test_input)
        if len(test_output) != len(test_input[0]):
            must_vectorize = True
    except:
        must_vectorize = True

    if must_vectorize:
        if warn_if_vectorizing:
            print(f"Warning: np.vectorize() is being called on the lambdified function {symbolic_expr}.")
        return np.vectorize(l)
    
    return l

In [ ]:
# Create callable functions for the model

# Implementation note:
# If u and v contain integrals (which happens if enhancement_factor is a function
# not a constant), then lambdify requires scipy as well as numpy and the result
# must be passed through np.vectorize to handle vector inputs, as described
# here: https://github.com/sympy/sympy/pull/20134#issuecomment-697336175

u_fn = lambdify_and_vectorize_if_needed((x, z), u, warn_if_vectorizing=True)
du_dx_fn = lambdify_and_vectorize_if_needed((x, z), du_dx, warn_if_vectorizing=True)
basal_velocity_fn = lambdify_and_vectorize_if_needed((x,), basal_velocity)
d_basal_velocity_dx_fn = lambdify_and_vectorize_if_needed((x,), sympy.diff(basal_velocity, x), warn_if_vectorizing=True)

# def dw_dz_ode(z, u, x):
#     return -1*du_dx_fn(x, z)

# def w_fn_nonvec(x, z):
#     sol = scipy.integrate.solve_ivp(dw_dz_ode, (0, z), [0], t_eval=[z],
#                                     args=(x,),
#                                     atol=1e-4/scipy.constants.year, rtol=1e-4/scipy.constants.year)
#     np.squeeze(sol.y)

# w_fn = np.vectorize(w_fn_nonvec)

def uw_on_grid(xs, zs, return_du_dx_fd=False):
    xs_extended = np.concatenate(([xs[0] - dx], xs, [xs[-1] + dx]))
    
    tstamp = datetime.datetime.now()

    du_dz_fn = sympy.lambdify((x, z), sympy.diff(u, z))

    def du_dz_ode(z, u, x):
        return du_dz_fn(x, z)
    
    sol = scipy.integrate.solve_ivp(du_dz_ode, (0, domain_z), basal_velocity_fn(xs_extended), t_eval=zs,
                                    args=(xs_extended,),
                                    atol=1e-4/scipy.constants.year, rtol=1e-4/scipy.constants.year)
    U_extended = np.transpose(sol.y)
    print(f"Shape of U_extended: {U_extended.shape}")
    print(f"Time to compute U_extended: {datetime.datetime.now() - tstamp}")
    tstamp = datetime.datetime.now()

    dudx_finite_diff = scipy.signal.correlate(U_extended, np.array([-1, 0, 1])[np.newaxis, :], mode='valid') / (2*(xs[1] - xs[0]))
    #dudx_finite_diff[:, :2] = np.nan
    #dudx_finite_diff[:, -2:] = np.nan
    W = np.cumsum(-1*dudx_finite_diff, axis=0) * (zs[1] - zs[0])

    U = U_extended[:, 1:-1]

    print(f"len(xs): {len(xs)}, len(zs): {len(zs)}")
    print(f"Shape of U: {np.shape(U)}")
    print(f"Shape of W: {np.shape(W)}")
    print(f"Time to compute W: {datetime.datetime.now() - tstamp}")

    if return_du_dx_fd:
        return U, W, dudx_finite_diff
    else:
        return U, W

# Create interpolators for U and W
X, Z = np.meshgrid(xs, zs)
U, W = uw_on_grid(xs, zs)
print(f"Time to compute U and W with numerical integration approach: {datetime.datetime.now() - tstamp}")

tstamp = datetime.datetime.now()
# Interpolated versions of U and W
U_interpolator = scipy.interpolate.RectBivariateSpline(xs, zs, np.transpose(U), bbox=[0, domain_x, 0, domain_z])
W_interpolator = scipy.interpolate.RectBivariateSpline(xs, zs, np.transpose(W), bbox=[0, domain_x, 0, domain_z])

U_interp = lambda x, z: U_interpolator(x, z, grid=False) # Call these functions, not the interpolators directly
W_interp = lambda x, z: W_interpolator(x, z, grid=False)

# U_interp = lambda x, z: scipy.interpolate.interpn((xs, zs), U.T, (x, z), bounds_error=False)
# W_interp = lambda x, z: scipy.interpolate.interpn((xs, zs), W.T, (x, z), bounds_error=False)
print(f"Time to create interpolators for U and W: {datetime.datetime.now() - tstamp}")

In [ ]:
# Plot the resulting horizontal and vertical velocity fields

below_surface = (Z < surface(X))
below_surface_mask = np.ones_like(X, dtype=float)
below_surface_mask[~below_surface] = np.nan

fig, (ax_U, ax_dWdz) = plt.subplots(2,1, figsize=(8, 6), sharex=True)
pcm_U = ax_U.pcolormesh(X/1e3, Z, scipy.constants.year*U*below_surface_mask, cmap='viridis', vmin=0)
fig.colorbar(pcm_U, ax=ax_U, label='Horizontal velocity [m/yr]')
ax_U.set_title('Horizontal velocity')
ax_U.set_ylabel('z [m]')
ax_U.set_ylim(0, domain_z)
pcm_W = ax_dWdz.pcolormesh(X/1e3, Z, scipy.constants.year*W*below_surface_mask, cmap='viridis', vmin=-20, vmax=2)
fig.colorbar(pcm_W, ax=ax_dWdz, label='Vertical velocity [m/yr]')
ax_dWdz.set_title('Vertical velocity')
ax_dWdz.set_xlabel('x [km]')
ax_dWdz.set_ylabel('z [m]')
fig.tight_layout()
fig.savefig(f"{output_results_base}_velocity_fields.png")
plt.show()

In [ ]:
# def compare_arrays(test, ref):
#     ref_is_zero = np.isclose(ref, 0, atol=1e-8)
#     diff = np.abs(test - ref)
#     print(f"Mean value of test where ref is zero: {np.nanmean(test[ref_is_zero])}")
#     print(f"Abs max value of test where ref is zero: {np.nanmax(np.abs(test[ref_is_zero]))}")

#     pct = np.abs(test - ref)/np.abs(ref)
#     print(f"Mean percent error (excluding where ref is zero): {np.nanmean(pct[~ref_is_zero])*100:.2f}%")
#     print(f"Max percent error (excluding where ref is zero): {np.nanmax(pct[~ref_is_zero])*100:.2f}%")

# print("== U comparison ==")
# compare_arrays(U_method2*below_surface_mask, U_method1*below_surface_mask)
# print("== W comparison ==")
# compare_arrays(W_method2*below_surface_mask, W_method1*below_surface_mask)

In [ ]:
# fig, (ax_Wsym, ax_Wfd, ax_Wdiff) = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
# pcm_Wsym = ax_Wsym.pcolormesh(X/1e3, Z, scipy.constants.year*W_method1*below_surface_mask, cmap='viridis')
# fig.colorbar(pcm_Wsym, ax=ax_Wsym, label='Vertical velocity [m/yr]')
# ax_Wsym.set_title('Vertical velocity (symbolic)')
# ax_Wsym.set_ylabel('z [m]')
# ax_Wsym.set_ylim(0, domain_z)
# pcm_Wfd = ax_Wfd.pcolormesh(X/1e3, Z, scipy.constants.year*W_method2*below_surface_mask, cmap='viridis')
# fig.colorbar(pcm_Wfd, ax=ax_Wfd, label='Vertical velocity [m/yr]')
# ax_Wfd.set_title('Vertical velocity (finite differences)')
# ax_Wfd.set_ylabel('z [m]')
# ax_Wfd.set_ylim(0, domain_z)
# pcm_Wdiff = ax_Wdiff.pcolormesh(X/1e3, Z, scipy.constants.year*(W_method1 - W_method2)*below_surface_mask, cmap='coolwarm', vmin=-1, vmax=1) 
# fig.colorbar(pcm_Wdiff, ax=ax_Wdiff, label='Vertical velocity difference [m/yr]')
# ax_Wdiff.set_title('Vertical velocity difference')
# ax_Wdiff.set_xlabel('x [km]')
# ax_Wdiff.set_ylabel('z [m]')
# fig.tight_layout()

In [ ]:
# For verification purposes, also plot the surface velocity

# Calculate the surface velocity (just below the surface to avoid border effects)
u_surface = lambdify_and_vectorize_if_needed(x, u.subs(z, surface_sym-0.1))(xs)
u_bed = lambdify_and_vectorize_if_needed(x, u.subs(z, 0))(xs)

# Plot the surface velocity
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xs/1e3, scipy.constants.year*u_surface, 'k-', label='Surface velocity [m/yr]')
ax.plot(xs/1e3, scipy.constants.year*u_bed, 'r-', label='Basal velocity [m/yr]')
#ax.set_title('Surface velocity')
ax.legend()
ax.set_xlabel('Distance [km]')
ax.set_ylabel('Horizontal Velocity [m/yr]')
ax.grid(True)
fig.savefig(f"{output_results_base}_surface_bed_velocity.png")
plt.show()

In [ ]:
# Vertical velocity at the bed

w_bed = lambdify_and_vectorize_if_needed(x, w.subs(z, 0))(xs)

# Plot the surface velocity
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xs/1e3, scipy.constants.year*w_bed, label='Basal velocity [m/yr]')
ax.legend()
ax.set_xlabel('Distance [km]')
ax.set_ylabel('Vertical Velocity [m/yr]')
ax.grid(True)
plt.show()

### Generate synthetic layers

In [ ]:
xs_layers = np.linspace(0, domain_x, 100) # don't need tight grid spacing for smooth layers

xs_layers_initial = xs_layers

# Layers from bottom starting flat and conforming to surface -- Used for rheology example
layers_t0 = []
for idx, start_offset in enumerate(np.arange(500, 2900, 200)):
    layer_start_fn = lambda x: start_offset + (idx/10)*(surface(x)-surface(0))
    layer = advect_layer(U_interp, W_interp,
                         xs_layers, layer_start_fn, [scipy.constants.year * 0])
    layers_t0.append(layer[-1])

In [ ]:
# Plot the advected layers on top of a pcolormesh plot of the velocity magnitude
# X, Z = np.meshgrid(xs, zs)
# U = sympy.lambdify((x, z), u)(X, Z)
# W = sympy.lambdify((x, z), w)(X, Z)
vel = np.sqrt(U**2 + W**2)

fig, ax = plt.subplots(figsize=(8, 4))
pcm = ax.pcolormesh(X/1e3, Z, scipy.constants.year * vel * below_surface_mask, cmap='viridis', alpha=0.8) #vmax=170)
fig.colorbar(pcm, ax=ax, label='Velocity magnitude [m/yr]')
for layer in layers_t0:
    ax.plot(xs_layers/1e3, layer(xs_layers), 'k--')
# for layer in layers_t1:
#     ax.plot(xs_layers/1e3, layer(xs_layers), 'r--')
#ax.set_title('Velocity magnitude and advected layers')
ax.set_title(f'n = {n}')
ax.grid(True)
ax.set_xlim(0, domain_x/1e3)
ax.set_ylim(0, surface(0))
ax.set_xlabel('x [km]')
ax.set_ylabel('z [m]')
fig.savefig(f"{output_results_base}_layers.png")
plt.show()

### Advect layers by 1 year to simulate layer motion

Current problem: somehow this now takes forever, even with enhancement_factor=1. Not sure why. Probably need to compare to prior versions.

In [ ]:
# rx = np.random.rand(1000)*domain_x
# rz = np.random.rand(1000)*domain_z

# u_lam = sympy.lambdify((x, z), u)
# u_lam_vec = np.vectorize(u_lam)
# w_lam = sympy.lambdify((x, z), w)
# w_lam_vec = np.vectorize(w_lam)

# print("U_interp:")
# %timeit U_interp(rx, rz)
# print("U_interp (non-vectorized calls):")
# %timeit [U_interp(x, z) for x, z in zip(rx, rz)]
# # print("lambdified u:")
# # %timeit u_lam(rx, rz)
# print("lambdified u (non-vectorized calls):")
# %timeit [u_lam(x, z) for x, z in zip(rx, rz)]
# print("vectorized lamdified u:")
# %timeit u_lam_vec(rx, rz)
# print("vectorized lamdified u (non-vectorized calls):")
# %timeit [u_lam_vec(x, z) for x, z in zip(rx, rz)]

# print("W_interp:")
# %timeit W_interp(rx, rz)
# print("W_interp (non-vectorized calls):")
# %timeit [W_interp(x, z) for x, z in zip(rx, rz)]
# # print("lambdified w:")
# # %timeit w_lam(rx, rz)
# print("lambdified w (non-vectorized calls):")
# %timeit [w_lam(x, z) for x, z in zip(rx, rz)]
# print("vectorized lamdified w:")
# %timeit w_lam_vec(rx, rz)
# print("vectorized lamdified w (non-vectorized calls):")
# %timeit [w_lam_vec(x, z) for x, z in zip(rx, rz)]

In [ ]:
xs_layers_t1 = np.arange(0, domain_x, 10)
layers_t1 = []
for layer in layers_t0:
    layers_t1.append(advect_layer(U_interp, W_interp,
                                  xs_layers_t1, layer, [scipy.constants.year * 1])[-1])

In [ ]:
layer_idx = 5
xs_layers_t1 = np.arange(0, domain_x, 10)

fig, ax = plt.subplots(figsize=(8, 4))
#ax.plot(xs_layers/1e3, layers_t0[layer_idx](xs_layers), 'k-', label='t=0')
# ax.plot(xs_layers/1e3, layers_t1[layer_idx](xs_layers), 'r-', label='t=1')
ax.plot(xs_layers_t1/1e3, layers_t1[layer_idx](xs_layers_t1) - layers_t0[layer_idx](xs_layers_t1), 'r-', label='t1 - t0')
#ax.plot(xs_layers_t1/1e3, layers_t1_measured[layer_idx](xs_layers_t1) - layers_t0_measured[layer_idx](xs_layers_t1), 'k--', label='t1 - t0 (with measurement noise)', alpha=0.5)
ax.set_title('Layer advection')
ax.legend()
ax.set_xlabel('x [km]')
ax.set_ylabel('z [m]')
ax.grid(True)
# ax.set_xlim(0, 3)
# ax.set_ylim(1820, 1860)

### Set up finite difference approximations of layer deformation-related partial derivatives

In [ ]:
# TODO: No noise for now

layers_t0_measured = layers_t0
layers_t1_measured = layers_t1

In [ ]:
# Create finite difference approximation functions for the simulated layer measurements
layer_dl_dx, layer_dl_dt, layer_d2l_dxdz, layer_d2l_dtdz = create_layer_finite_difference_fns(layers_t0_measured, layers_t1_measured)


In [ ]:
# Plot the advected layers on top of a pcolormesh plot of the velocity magnitude
U, W = uw_on_grid(xs, zs)
vel = np.sqrt(U**2 + W**2)

fig, ax = plt.subplots(figsize=(8, 4))
#pcm = ax.pcolormesh(X/1e3, Z, scipy.constants.year * vel * below_surface_mask, cmap='grey', alpha=0.5)
vmin, vmax = -2, 2
sc = None
for idx, layer in enumerate(layers_t0):
    sc = ax.scatter(xs/1e3, layer(xs), c=np.arctan(layer_dl_dx(xs, idx))*(180/np.pi), s=2, vmin=vmin, vmax=vmax, cmap='coolwarm')
fig.colorbar(sc, ax=ax, label='Layer slope [deg]')
ax.set_title('Layer slope')
ax.grid(True)
ax.set_xlim(0, domain_x/1e3)
ax.set_ylim(0, surface(0))
ax.set_xlabel('x [km]')
ax.set_ylabel('z [m]')
fig.savefig(f"{output_results_base}_layer_slope.png")
plt.show()

### Method of Characteristics Solution

In [ ]:
start_pos_x = 100
layer_solutions = solve_all_layers(layers_t0, layer_d2l_dxdz, layer_d2l_dtdz, u, x, z, domain_x, xs_layers, start_pos_x=start_pos_x)

In [ ]:
def layer_solution_velocity(layer_idx, x):
    res = layer_solutions[layer_idx].sol(x)[0]
    if np.isscalar(res):
        if x > layer_solutions[layer_idx].sol.t_max:
            return np.nan
        else:
            return res
    res[x > layer_solutions[layer_idx].sol.t_max] = np.nan
    return res

In [ ]:
fig, (ax, ax_err) = plt.subplots(2, 1, figsize=(8, 8), sharex=True, sharey=True)
sc = None

# Solution plot
for layer_idx in layer_solutions.keys():
    xs_tmp = xs
    sc = ax.scatter(xs_tmp/1e3, layers_t0[layer_idx](xs_tmp), c=layer_solution_velocity(layer_idx, xs_tmp), vmin=0, vmax=40, s=2, cmap='viridis')
fig.colorbar(sc, ax=ax, label='Horizontal Velocity [m/yr]')

ax.grid(True)
ax.set_xlim(0, domain_x/1e3)
ax.set_ylim(0, surface(0))
#ax.set_xlabel('x [km]')
ax.set_ylabel('z [m]')

# Error plot

for layer_idx in layer_solutions.keys():
    xs_tmp = xs
    err = layer_solution_velocity(layer_idx, xs_tmp) - (lambdify_and_vectorize_if_needed((x,z), u)(xs_tmp, layers_t0[layer_idx](xs_tmp)) * scipy.constants.year)
    sc = ax_err.scatter(xs_tmp/1e3, layers_t0[layer_idx](xs_tmp), c=err, vmin=-2, vmax=2, s=2, cmap='coolwarm_r')
fig.colorbar(sc, ax=ax_err, label='Horizontal Velocity Error [m/yr]')

ax_err.grid(True)
ax_err.set_xlabel('x [km]')
ax_err.set_ylabel('z [m]')

ax.set_title('Horizontal velocity solution')
ax_err.set_title('Error in horizontal velocity estimate')
fig.savefig(f"{output_results_base}_layer_solutions.png")
plt.show()

In [ ]:
# Plot each layer solution at x=100e3 as a function of depth
fig, (ax_u, ax_dwdz) = plt.subplots(1,2, figsize=(8, 8), sharey=True)

plot_pos_x = 80e3

u_at_plot_pos = lambdify_and_vectorize_if_needed(z, u.subs(x, plot_pos_x))(zs)

for layer_idx in layer_solutions.keys():
    if layer_idx == 1:
        lbl = 'ODE Solutions'
    else:
        lbl = None
    ax_u.scatter([layer_solution_velocity(layer_idx, plot_pos_x)], [layers_t0[layer_idx](plot_pos_x)], label=lbl, c='r')

ax_u.plot(u_at_plot_pos*scipy.constants.year, zs, 'k--', label='True')
ax_u.set_title(f'Horizontal Velocity\nn = {n}')
ax_u.set_xlabel('Horizontal velocity [m/yr]')
ax_u.set_ylabel('z [m]')
ax_u.grid()
ax_u.legend()
ax_u.set_xlim(0, 1.2 * np.nanmax(u_at_plot_pos*scipy.constants.year))

# Vertical strain rate
dwdz_at_plot_pos = lambdify_and_vectorize_if_needed(z, sympy.diff(w, z).subs(x, plot_pos_x))(zs)

for layer_idx in layer_solutions.keys():
    if layer_idx == 1:
        lbl_ode = 'ODE Solutions'
        lbl_zeroapprox = 'Zero Slope Approximation'
    else:
        lbl_ode = None
        lbl_zeroapprox = None
    
    # Recovered from layer ODE solutions
    du_dx_ode_at_plot_pos = layer_solution_velocity(layer_idx, plot_pos_x) - layer_solution_velocity(layer_idx, plot_pos_x-1)
    ax_dwdz.scatter([-1*du_dx_ode_at_plot_pos], [layers_t0[layer_idx](plot_pos_x)], label=lbl_ode, c='r')

    # Estimated from zero slope approximation
    dwdz_zs_approx = (layer_dl_dt(plot_pos_x, layer_idx+1) - layer_dl_dt(plot_pos_x, layer_idx-1)) / (layers_t0[layer_idx+1](plot_pos_x) - layers_t0[layer_idx-1](plot_pos_x))
    ax_dwdz.scatter([dwdz_zs_approx], [layers_t0[layer_idx](plot_pos_x)], label=lbl_zeroapprox, c='b', marker='x')

ax_dwdz.plot(dwdz_at_plot_pos*scipy.constants.year, zs, 'k--', label='True')
ax_dwdz.set_title(f'Vertical Strain Rate\nn = {n}')
ax_dwdz.set_xlabel('Vertical strain rate [m/(yr*m)]')
ax_dwdz.grid()
ax_dwdz.legend()

fig.savefig(f"{output_results_base}_layer_solutions_at_xpos.png")
plt.show()

### Estimate effective stress

In [ ]:
rho = 918
g = 9.8

visc_start_x = 5e3
visc_end_x = 95e3

xs_visc = np.linspace(visc_start_x, visc_end_x, 100)

# Estimate du_dz and effective viscosity for the entire domain
du_dz_central_diff = np.zeros((len(xs_visc), len(layer_solutions)-1))
eff_stress = np.zeros_like(du_dz_central_diff)

for idx, x_pos in enumerate(xs_visc):
    for layer_idx in np.arange(2, len(layers_t0)-2):
        du_dz_central_diff[idx, layer_idx-1] = (layer_solution_velocity(layer_idx-1, x_pos) - layer_solution_velocity(layer_idx+1, x_pos)) / (layers_t0[layer_idx-1](x_pos) - layers_t0[layer_idx+1](x_pos))
        dist_to_surf = surface(x_pos) - layers_t0[layer_idx](x_pos)
        eff_stress[idx, layer_idx-1] = rho * g * dist_to_surf * -1 * ds_dx(x_pos)

# Plot a scatter plot of log10(du_dz_central_diff) vs log10(eff_stress)
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(np.log10(eff_stress), np.log10(du_dz_central_diff), s=2, label=f'n = {n}')
ax.set_aspect('equal')
ax.set_xlabel('log(effective stress)')
ax.set_ylabel('log(strain rate)')
ax.legend()
ax.grid()
fig.savefig(f"{output_results_base}_stress_strain_rate.png")
plt.show()

### Save results to a pickle

In [ ]:
# Create filename containing current timestamp and n value
filename = output_results_base + ".pickle"
with open(filename, 'wb') as f:
    pickle.dump({
        'n': n,
        'xs': xs,
        'zs': zs,
        'domain_x': domain_x,
        'domain_z': domain_z,
        'surface': surface_sym,
        'x': x,
        'z': z,
        'u': u,
        'w': w,
        'ds_dx': ds_dx_sym,
        'layers_t0': layers_t0,
        'layers_t1': layers_t1,
        'layer_solutions': layer_solutions,
        'eff_stress': eff_stress,
        'du_dz_central_diff': du_dz_central_diff
    }, f)

print(f"Saved to {filename}")

### Find vertical velocity from horizontal velocity

In [ ]:
# Zero slope approximation vertical strain rate

xs_tmp = xs[xs > start_pos_x]

interp_x = np.zeros((len(layer_solutions), len(xs_tmp)))
interp_z = np.zeros_like(interp_x)
interp_dl_dt = np.zeros_like(interp_x)
interp_dl_dx = np.zeros_like(interp_x)
for idx, layer_idx in enumerate(layer_solutions.keys()):
    interp_x[idx, :] = xs_tmp
    interp_z[idx, :] = layers_t0[layer_idx](xs_tmp)
    interp_dl_dt[idx, :] = layer_dl_dt(xs_tmp, layer_idx)
    interp_dl_dx[idx, :] = layer_dl_dx(xs_tmp, layer_idx)

X, Z = np.meshgrid(xs_tmp, zs)
dl_dt_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()), interp_dl_dt.flatten(), (X, Z), method='linear')
dl_dx_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()), interp_dl_dx.flatten(), (X, Z), method='linear')
dw_dz_zeroslope_grid = np.gradient(dl_dt_grid, zs, axis=0)


In [ ]:
# Interpolate layer solutions to a grid defined by xs and zs
xs_tmp = xs[xs > start_pos_x]
interp_x = np.zeros((len(layer_solutions), len(xs_tmp)))
interp_z = np.zeros_like(interp_x)
interp_u = np.zeros_like(interp_x)
interp_w = np.zeros_like(interp_x)
for idx, layer_idx in enumerate(layer_solutions.keys()):
    interp_x[idx, :] = xs_tmp
    interp_z[idx, :] = layers_t0[layer_idx](xs_tmp)
    interp_u[idx, :] = layer_solution_velocity(layer_idx, xs_tmp)

    interp_w[idx, :] = layer_dl_dt(xs_tmp, layer_idx) + interp_u[idx,:]*layer_dl_dx(xs_tmp, layer_idx)

# Gridded horizontal velocity from layer line solutions
X, Z = np.meshgrid(xs_tmp, zs)
u_mol_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()),
                                        interp_u.flatten(), (X, Z), method='linear')
w_mol_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()),
                                        interp_w.flatten(), (X, Z), method='linear')

# Mask inside non-outer layers
mask = np.nan * np.zeros_like(X)
l1 = layers_t0[1](xs_tmp)
l2 = layers_t0[-2](xs_tmp)
mask[Z < np.maximum(l1, l2)] = 1
mask[Z <= np.minimum(l1, l2)] = np.nan

# Vertical strain rate
dw_dz_mol_grid = -1 * np.gradient(u_mol_grid, xs_tmp, axis=1)

fig, axs = plt.subplots(5,2, figsize=(16, 12), sharex=True)
((ax_U, ax_U_err), (ax_W, ax_W_err), (ax_W_zeroslope, ax_W_zeroslope_err), (ax_dWdz, ax_dWdz_err), (ax_dWdz_zeroslope, ax_dWdz_zeroslope_err)) = axs

err_clb_pct_of_max = 0.1

# Get the true horizontal and vertical velocities
U_tmp, W_tmp, dudx_finite_diff = uw_on_grid(xs_tmp, zs, return_du_dx_fd=True)

# dw_dz_sym = sympy.diff(scipy.constants.year * w, z)
# dw_dz_lambdify = lambdify_and_vectorize_if_needed((x, z), dw_dz_sym)
# dw_dz_true = dw_dz_lambdify(X, Z)

dw_dz_true = -1 * dudx_finite_diff * scipy.constants.year


# Horizontal velocity

pcm_U = ax_U.pcolormesh(X/1e3, Z, mask * u_mol_grid, cmap='viridis')
fig.colorbar(pcm_U, ax=ax_U, label='Horizontal velocity [m/yr]')
ax_U.set_title('Horizontal velocity\n(interpolated from layer ODEs)')
ax_U.set_ylabel('z [m]')

pcm_U_err = ax_U_err.pcolormesh(X/1e3, Z, mask * (u_mol_grid - U_tmp*scipy.constants.year), cmap='coolwarm',
                                vmin=-1*err_clb_pct_of_max*np.max(U_tmp*scipy.constants.year),
                                vmax=err_clb_pct_of_max*np.max(U_tmp*scipy.constants.year))
fig.colorbar(pcm_U_err, ax=ax_U_err, label='Error in horizontal velocity [m/yr]')
ax_U_err.set_title('Error in horizontal velocity\n(ODE interpolation - true)')

# Vertical velocity

pcm_W = ax_W.pcolormesh(X/1e3, Z, mask * w_mol_grid, cmap='viridis')
fig.colorbar(pcm_W, ax=ax_W, label='Vertical velocity [m/yr]')
ax_W.set_title('Vertical velocity\n(interpolated from layer ODEs)')
ax_W.set_ylabel('z [m]')

pcm_W_err = ax_W_err.pcolormesh(X/1e3, Z, mask * (w_mol_grid - W_tmp*scipy.constants.year), cmap='coolwarm',
                                vmin=-1*err_clb_pct_of_max*np.max(np.abs(W_tmp*scipy.constants.year)),
                                vmax=err_clb_pct_of_max*np.max(np.abs(W_tmp*scipy.constants.year)))
fig.colorbar(pcm_W_err, ax=ax_W_err, label='Error in vertical velocity [m/yr]')
ax_W_err.set_title('Error in vertical velocity\n(ODE interpolation - true)')

# Vertical velocity (zero layer slope approximation)

vmin_ref, vmax_ref = pcm_W.get_clim()
pcm_W_zeroslope = ax_W_zeroslope.pcolormesh(X/1e3, Z, mask * dl_dt_grid, cmap='viridis', vmin=vmin_ref, vmax=vmax_ref)
fig.colorbar(pcm_W_zeroslope, ax=ax_W_zeroslope, label='Vertical velocity [m/yr]')
ax_W_zeroslope.set_title('Vertical velocity\n(zero slope approximation)')
ax_W_zeroslope.set_ylabel('z [m]')

vmin_ref, vmax_ref = pcm_W_err.get_clim()
pcm_W_zeroslope_err = ax_W_zeroslope_err.pcolormesh(X/1e3, Z, mask * (dl_dt_grid - W_tmp*scipy.constants.year), cmap='coolwarm', vmin=vmin_ref, vmax=vmax_ref)
fig.colorbar(pcm_W_zeroslope_err, ax=ax_W_zeroslope_err, label='Error in vertical velocity [m/yr]')
ax_W_zeroslope_err.set_title('Error in vertical velocity\n(zero slope approximation - true)')

# Vertical strain rate

dwdz_abs_max = np.nanmax(np.abs(mask * dw_dz_mol_grid))
pcm_dWdz = ax_dWdz.pcolormesh(X/1e3, Z, mask * dw_dz_mol_grid, cmap='coolwarm', vmin=-1*dwdz_abs_max, vmax=dwdz_abs_max)
fig.colorbar(pcm_dWdz, ax=ax_dWdz, label='Vertical strain rate [m/(yr*m)]')
ax_dWdz.set_title('Vertical strain rate\n(interpolated from layer ODEs)')
ax_dWdz.set_xlabel('x [km]')
ax_dWdz.set_ylabel('z [m]')



vmin_ref, vmax_ref = pcm_dWdz.get_clim()
dwdz_abs_max = np.maximum(np.abs(vmin_ref), np.abs(vmax_ref))
pcm_dWdz_err = ax_dWdz_err.pcolormesh(X/1e3, Z, mask * (dw_dz_mol_grid - dw_dz_true), cmap='coolwarm', vmin=-0.1*dwdz_abs_max, vmax=0.1*dwdz_abs_max)
fig.colorbar(pcm_dWdz_err, ax=ax_dWdz_err, label='Error in vertical strain rate [m/(yr*m)]')
ax_dWdz_err.set_title('Error in vertical strain rate\n(ODE interpolation - true)')
ax_dWdz_err.set_xlabel('x [km]')
ax_dWdz_err.set_ylabel('z [m]')

# Vertical strain rate (zero slope approximation)

vmin_ref, vmax_ref = pcm_dWdz.get_clim()
pcm_dWdz_zeroslope = ax_dWdz_zeroslope.pcolormesh(X/1e3, Z, mask * dw_dz_zeroslope_grid, cmap='coolwarm', vmin=vmin_ref, vmax=vmax_ref)
fig.colorbar(pcm_dWdz_zeroslope, ax=ax_dWdz_zeroslope, label='Vertical strain rate [m/(yr*m)]')
ax_dWdz_zeroslope.set_title('Vertical strain rate\n(zero slope approximation)')
ax_dWdz_zeroslope.set_xlabel('x [km]')
ax_dWdz_zeroslope.set_ylabel('z [m]')

vmin_ref, vmax_ref = pcm_dWdz_err.get_clim()
pcm_dWdz_zeroslope_err = ax_dWdz_zeroslope_err.pcolormesh(X/1e3, Z, mask * (dw_dz_zeroslope_grid - dw_dz_true), cmap='coolwarm', vmin=vmin_ref, vmax=vmax_ref)
fig.colorbar(pcm_dWdz_zeroslope_err, ax=ax_dWdz_zeroslope_err, label='Error in vertical strain rate [m/(yr*m)]')
ax_dWdz_zeroslope_err.set_title('Error in vertical strain rate\n(zero slope approximation - true)')
ax_dWdz_zeroslope_err.set_xlabel('x [km]')
ax_dWdz_zeroslope_err.set_ylabel('z [m]')

# Draw layer lines on all plots for reference
for layer in layers_t0:
    for ax in axs.flatten():
        ax.plot(xs_layers/1e3, layer(xs_layers), 'k--', linewidth=0.5)

fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(16, 4), sharex=True)
ax_U, ax_U_pct_surf = axs

# Horizontal velocity

pcm_U = ax_U.pcolormesh(X/1e3, Z, mask * u_mol_grid, cmap='viridis')
fig.colorbar(pcm_U, ax=ax_U, label='Horizontal velocity [m/yr]')
ax_U.set_title('Horizontal velocity\n(interpolated from layer ODEs)')
ax_U.set_ylabel('z [m]')

surface_u = lambdify_and_vectorize_if_needed(x, u.subs(z, surface_sym-0.1))(xs_tmp) * scipy.constants.year

pcm_U_pct_surf = ax_U_pct_surf.pcolormesh(X/1e3, Z, mask * u_mol_grid / surface_u, cmap='viridis', vmax=1, vmin=0.6)
fig.colorbar(pcm_U_pct_surf, ax=ax_U_pct_surf, label='Horizontal velocity / Surface velocity [diml]')


# Draw layer lines on all plots for reference
for layer in layers_t0:
    for ax in axs.flatten():
        ax.plot(xs_layers/1e3, layer(xs_layers), 'k--', linewidth=0.5)

fig.tight_layout()
plt.show()

### Explore a single layer solution

In [ ]:
layer_idx = 1

fig, axs = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
ax_U, ax_W, ax_stab = axs

sol = layer_solutions[layer_idx]

# Horizontal Velocity
ax_U.set_title(f'Horizontal Velocity at Layer {layer_idx}')
ax_U.scatter(sol.t/1e3, sol.y, s=2, label=f'ODE solution for layer {layer_idx}', c='red')
# truth
xs_tmp = sol.t
ax_U.plot(xs_tmp/1e3, lambdify_and_vectorize_if_needed((x,z), u)(xs_tmp, layers_t0[layer_idx](xs_tmp)) * scipy.constants.year, 'k--', alpha=0.5, label='True')

# Vertical Velocity
ax_W.set_title(f'Vertical Velocity at Layer {layer_idx}')
ax_W.scatter(sol.t/1e3, layer_dl_dt(sol.t, layer_idx) + sol.y[0,:]*layer_dl_dx(sol.t, layer_idx), s=2, label='w(x)')
ax_W.plot(xs_tmp/1e3, lambdify_and_vectorize_if_needed((x,z), w)(xs_tmp, layers_t0[layer_idx](xs_tmp)) * scipy.constants.year, 'k--', alpha=0.5, label='True')

ax_W.scatter(sol.t/1e3, layer_dl_dt(sol.t, layer_idx), s=2, label='dl/dt')
ax_W.scatter(sol.t/1e3, sol.y[0,:]*layer_dl_dx(sol.t, layer_idx), s=2, label='u * dl/dx')

ax_W.set_ylim(-4,4)

# # Plot the derivative
# ax_deriv.set_title(f'du/dtau at Layer {layer_idx}')
# xs_tmp = sol.t
# ax_deriv.scatter(xs_tmp/1e3, du_dtau(xs_tmp, sol.y[0,:], layer_idx), s=2, label="du_dtau(x) (ODE Function)")

# Stability criterion
ax_stab.set_title(f'd2l_dxdz at Layer {layer_idx} (stability)')
stab_crit = layer_d2l_dxdz(sol.t, layer_idx)
ax_stab.scatter(sol.t/1e3, stab_crit, s=2)

# Properties for all plots
for ax in axs:
    ax.grid(True)
    ax.legend()
    ax.set_xlim(0, domain_x/1e3)

plt.show()

In [ ]:
print(f"Total elapsed time: {(time.time() - t_start_notebook)/60:.2f} minutes")